# LeNet 0730

In [27]:
import numpy as np
from pynq import Xlnk
from pynq import Overlay

In [28]:
overlay = Overlay("./lenet.bit")

In [29]:
dma = overlay.axi_dma_0

In [30]:
lenet = overlay.lenet_0

### AXI Stream 传输
```c++
struct wide_stream {
	float data;
	ap_uint<1> user;
	ap_uint<1> last;
};
```
数据传输格式：float  32bit

故

`input_array.dtpye=np.float32`

`output_array.dtpye=np.float32`

`in_buffer.dtype=np.float32`

`out_buffer.dtype=np.float32`

In [31]:
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(62494), 
                           dtype=np.float32)
out_buffer = xlnk.cma_array(shape=(1), 
                            dtype=np.float32)

### Parameter 存放格式
**image**和**parameter**存放在`input_array`中
> iamge 32x32=1024

> layer1 weights  5x5x6 = 25x6 = 150

> layer3 weights  5x5x6x16 = 25x6x16 =2400

> layer5 weights 400x120 = 48000

> layer6 weights 84x120 = 10080

> layer7 weights 10x84 = 840

> 1024+150+2400+48000+10080+840=62494

- input_array[0:149] = layer1 weights    **[Output Channel,Row,Column]**
- input_array[150:2549] = layer3 weights    **[Output Channel,Input Channel,Row,Column]**
- input_array[2550:50549] = layer5 weights    **[Output Channel,Input Channel]**
- input_array[50550:60629] = layer6 weights      **[Output Channel,Input Channel]**
- input_array[60630:61469] = layer7 weights      **[Output Channel,Input Channel]**
- input_array[61470:62493] = image[0,0],image[0,1]...image[0,31],image[1,0]...image[31,31]     **[row,column]**

In [32]:
input_array = np.arange(0,62494,1,dtype=np.float32) #just for test

In [33]:
import pickle

In [34]:
with open('input_array_attack.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    input_array = pickle.load(f)

In [35]:
input_array.shape

(62494,)

In [36]:
output_array = np.empty((1,),dtype=np.float32)

In [37]:
np.copyto(in_buffer,input_array)

In [38]:
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)    
lenet.write(0x00,0x81) # start

In [39]:
dma.sendchannel.wait()

In [40]:
dma.recvchannel.wait()

In [41]:
np.copyto(output_array,out_buffer)

In [42]:
print(output_array)

[ 4.]
